## Handling incoming data from oanda

For any trading class, we have to take in input data and then use that to make decisions. Here, demonstrating how we can get information and save it down to the class

In [1]:
import pandas as pd 
import numpy as np 
import tpqoa

In [23]:
class Trader(tpqoa.tpqoa): 

    def __init__(self):
        super().__init__('oanda.cfg')
        self._data = pd.DataFrame()

    def on_success(self, time, bid, ask): 
        print("Hit")
        bid_ask_spread = (ask - bid)
        mid = bid + (bid_ask_spread) / 2
        df = pd.DataFrame([[time, bid, ask, mid, bid_ask_spread]], columns = ['Time', 'Bid', 'Ask', 'Mid', 'Spread'])
        df.set_index('Time', inplace = True)
        df.index = pd.to_datetime(df.index)
        self._data = pd.concat([self._data, df])

    def get_historical_data(self): 
        return self._data

    
        
    


In [24]:
sample = Trader()

In [ ]:
sample.stream_data(instrument='GBP_USD', stop = 100)

In [26]:
data = sample.get_historical_data()

In [6]:
# data = pd.read_csv("test_data/sample_tick_data.csv", parse_dates = True)
# data.set_index('Time', inplace = True)
# data.index = pd.to_datetime(data.index)

In [31]:
resampled = data.resample('10S')

In [32]:
h = resampled.max().fillna(method = 'ffill')
l = resampled.min().fillna(method = 'ffill')
o = resampled.first().fillna(method = 'ffill')
c = resampled.last().fillna(method = 'ffill')

In [33]:
ohlc = pd.concat([o['Mid'], h['Mid'], l['Mid'], c['Mid']], axis = 1)
ohlc.columns = ['Open', 'High', 'Low', 'Close']

In [34]:
ohlc

,Open,High,Low,Close
Time,,,,
2022-07-24 23:59:50+00:00,1.198565,1.198565,1.198565,1.198565
2022-07-25 00:00:00+00:00,1.198585,1.198585,1.198450,1.198480
2022-07-25 00:00:10+00:00,1.198485,1.198550,1.198400,1.198405
2022-07-25 00:00:20+00:00,1.198425,1.198445,1.198365,1.198370
2022-07-25 00:00:30+00:00,1.198370,1.198500,1.198370,1.198475
2022-07-25 00:00:40+00:00,1.198480,1.198595,1.198465,1.198570
